In [ ]:
import pandas as pd
import numpy as np
import requests

In [ ]:
df = pd.read_csv('spam.csv' , encoding='latin-1')

In [ ]:
df

In [ ]:
## taking only the first and second columns :
df = df[['v1', 'v2']]
df.head()                                     ## print  first five tuples

In [ ]:
## Converting value of v1 in numeric
df['v1'] = df['v1'].apply(lambda x : 1 if x=='ham' else 0)
df.head()

In [ ]:
from tensorflow.keras.models import Sequential                                ### importing required packages
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding
from tensorflow.keras import metrics
from tensorflow.keras.preprocessing import sequence

In [ ]:
tag = df["v1"].to_list()                   ### converting pandas series to list
text = df["v2"].to_list()

In [ ]:
for ix,x in enumerate(text):                         ## spliting in tockens
  text[ix] = x.split(' ')

In [ ]:
vocab = {}           ### dictionary that will contain word : numeric_code

for x in text:
  for y in x:
    if y not in vocab.keys():
      vocab[y] = len(vocab)+1

In [ ]:
len(vocab)          ## Total unique tokens

In [ ]:
#### converting the text to numerics
for x in text:
  for iy,y in enumerate(x):
    x[iy] = vocab[y]

In [ ]:
### Making all the texts of equal length i.e. 100 words.
### trimming the larger sentences to 100 words and adding a dummy 0
X = sequence.pad_sequences(np.array(text),maxlen=100 , value = 0 , padding = 'post')
y = np.array(tag)     ## converting to np array

In [ ]:
print(X.shape , y.shape )             ## printing the shape of x and y

In [ ]:
from sklearn.model_selection import train_test_split                         ## to split the dataset in training and testing sets
x_train,x_test,y_train,y_test = train_test_split(X,y, test_size = 0.2)

In [ ]:
max_features = 20000                        ## max vocab size

In [ ]:
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2,activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc'] )


In [ ]:
model.summary()

In [ ]:
model.fit(x_train , y_train, batch_size=128 , epochs =2 , validation_data= (x_test,y_test), validation_batch_size=32)

In [ ]:
def predict_sms(txt):       #### a function to predict users input sentence, make sure that trained model is laded in model variable
  txt = txt.split(' ')
  for ix,x in enumerate(txt):
    try:
      z  = vocab[x]
    except:
      z = 0
    txt[ix] = z
  #print(txt)
  txt = sequence.pad_sequences([txt],maxlen=100 , value = 0 , padding = 'post')
  txt = model.predict(txt)
  #print('Output probability of model -' , txt[0][0] )
  if txt[0][0] >= 0.5 :
    return "Its a ham."
  else:
    return "Its a spam."

In [ ]:
predict_sms('Who are you?')